In [1]:
from data import Data
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
import itertools
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
d = Data(first=2000, x_transpose=(0, 3, 1, 2))
X_train,y_train = d.get_train()
X_cross, y_cross = d.get_dev()
X_test,y_test = d.get_test()

loaded 100
loaded 200
loaded 300
loaded 400
loaded 500
loaded 600
loaded 700
loaded 800
loaded 900
loaded 1000
loaded 1100
loaded 1200
loaded 1300
loaded 1400
loaded 1500
loaded 1600
loaded 1700
loaded 1800
loaded 1900
loaded 2000
loaded 2100
loaded 2200
loaded 2300
loaded 2400
loaded 2500
loaded 2600
loaded 2700
loaded 2800
loaded 2900
loaded 3000
loaded 3100
loaded 3200
loaded 3300
loaded 3400
loaded 3500
loaded 3600
loaded 3700
loaded 3800
loaded 3900
loaded 4000


In [3]:
y_train = y_train
print(X_train.shape)
print(y_train.shape)
print(X_cross.shape)
print(y_cross.shape)
print(X_test.shape)
print(y_test.shape)

(2137, 3, 224, 224)
(2137,)
(658, 3, 224, 224)
(658,)
(494, 3, 224, 224)
(494,)


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader
from torch.utils.data import sampler
import torch.nn.functional as F
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
import torchvision.models as models

dtype = torch.float

USE_GPU = True
if USE_GPU:
    dtypeFloat = torch.cuda.FloatTensor
    dtypeLong = torch.cuda.LongTensor
    device = torch.device('cuda')
else:
    dtypeFloat = torch.FloatTensor
    dtypeLong = torch.LongTensor
    device = torch.device('cpu')

In [5]:
def flatten(X):
    N = X.shape[0]
    return X.view(N,-1)

In [6]:
class Flatten(nn.Module):
    def forward(self, x): 
        return flatten(x)

In [7]:
class MyCustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
    def __getitem__(self, index):
        img = self.X[index]
        label = self.y[index]
        return (img, label)

    def __len__(self):
        return self.X.shape[0]

In [48]:
batch_size = 20
NUM_TRAIN = X_train.shape[0]
train_dataset = MyCustomDataset(X_train, y_train)
loader_train = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)), drop_last = True)

train_dataset_check = MyCustomDataset(X_train, y_train)
loader_train_check = DataLoader(train_dataset_check, batch_size=150, sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)), drop_last = True)

NUM_CROSS = X_cross.shape[0]
cross_dataset = MyCustomDataset(X_cross, y_cross)
loader_cross = DataLoader(cross_dataset, batch_size=batch_size, sampler=sampler.SubsetRandomSampler(range(NUM_CROSS)), drop_last = True)

cross_dataset_check = MyCustomDataset(X_cross, y_cross)
loader_cross_check = DataLoader(cross_dataset_check, batch_size=150, sampler=sampler.SubsetRandomSampler(range(NUM_CROSS)), drop_last = True)

In [49]:
def check_accuracy(loader, model): 
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            scores = model(Variable(x.float().type(dtypeFloat)))
            _, preds = scores.max(1)
            num_correct += (preds == Variable(y.long().type(dtypeLong))).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

In [50]:
def imshow_noax(img, normalize=True):
    """ Tiny helper to show images as uint8 and remove axis labels """
    print(img.shape)
    img = img.transpose(1,2,0)
    if normalize:
        img_max, img_min = np.max(img), np.min(img)
        img = 255.0 * (img - img_min) / (img_max - img_min)
    plt.imshow(img.astype('uint8'))
    plt.gca().axis('off')

In [51]:
print_every = 5
print_acc_every = 50
show_transformations = False

def train(m, optimizer, epochs=15):
    loss_arr = []
    for e in range(epochs):
        for t, (x, y) in enumerate(loader_train):
            m.train()
            scores = m(Variable(x.float().type(dtypeFloat)))
            loss = F.cross_entropy(scores, Variable(y.long().type(dtypeLong)))
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            if t % print_every == 0:
                print('Iteration %d, loss = %.4f' % (t, loss))
                loss_arr += [loss.item()]
                if (t % print_acc_every == 0):
                    print ("train acc:")
                    check_accuracy(loader_train_check, m)
                    print ("cross acc:")
                    m.eval()
                    check_accuracy(loader_cross_check, m)
                    
                    # print transformations
                    if show_transformations:
                        x_ = x[5][None]
                        stn = next(m.modules())[0]
                        plt.subplot(1, 2, 1)
                        i = 50
                        imshow_noax(x_.data.numpy()[0], normalize=False)
                        plt.subplot(1, 2, 2)
                        stn_out = stn(Variable(x_.float().type(dtypeFloat))).data.numpy()[0]
                        imshow_noax(stn_out, normalize=False)
                        plt.show()
                    
    return loss_arr

In [52]:
attn_channel_1 = 96
attn_channel_2 = 128
attn_channel_3 = 256
n_class=2

class FeatureExtraction(nn.Module):
    def __init__(self):
        super(FeatureExtraction, self).__init__()
        
        self.model = nn.Sequential(            
            nn.Conv2d(3, attn_channel_1, 7, stride=1, padding=3), # 224 x 224 x 96
            nn.MaxPool2d(2, stride=2), # 112 x 112 x 96
            nn.ReLU(),
            nn.Conv2d(attn_channel_1, attn_channel_2, 5, stride=1, padding=2), # 112 x 112 x 128
            nn.MaxPool2d(2, stride=2), # 56 x 56 x 128
            nn.ReLU(),
            nn.Conv2d(attn_channel_2, attn_channel_3, 3, stride=1, padding=1), # 56 x 56 x 256
            nn.MaxPool2d(2, stride=2), # 28 x 28 x 256
            nn.ReLU(),
        )       
        
    def forward(self, x):
        return self.model(x)

In [53]:
n_class = 2
C, H, W = 256, 28, 28

class MapToClass(nn.Module):
    def __init__(self):
        super(MapToClass, self).__init__()
        
        self.model = nn.Sequential(            
            Flatten(),
            nn.Linear(C * H * W, n_class),
        )       
        
    def forward(self, x):
        return self.model(x)        

In [54]:
class ProduceSaliency(nn.Module):
    def __init__(self):
        super(ProduceSaliency, self).__init__()
        
        self.model = nn.Sequential(            
            nn.Conv2d(C, 1, 1, stride=1, padding=0)
        )       
        
    def forward(self, x):
        return self.model(x)   

In [58]:
filter_size=5
C, H, W = 256, 28, 28

class AttentiveConvLSTM(nn.Module):
    def __init__(self):
        super(AttentiveConvLSTM, self).__init__()
        
        self.Tanh = nn.Tanh()
        self.Sigmoid = nn.Sigmoid()
        
        self.Conv_Wa = nn.Conv2d(C, C, filter_size, padding=2)
        self.Conv_Ua = nn.Conv2d(C, C, filter_size, padding=2)
        self.Conv_Va = nn.Conv2d(C, 1, filter_size, padding=2)
        self.SoftMax_a = nn.Softmax(dim=2)
        self.ba = Variable(torch.randn(H, device = device, dtype=dtype, requires_grad=True))
        
        self.Conv_Wi = nn.Conv2d(C, C, filter_size, padding=2)
        self.Conv_Ui = nn.Conv2d(C, C, filter_size, padding=2)
        self.bi = Variable(torch.randn(H, device = device, dtype=dtype, requires_grad=True))
    
        self.Conv_Wf = nn.Conv2d(C, C, filter_size, padding=2)
        self.Conv_Uf = nn.Conv2d(C, C, filter_size, padding=2)
        self.bf = Variable(torch.randn(H, device = device, dtype=dtype, requires_grad=True))

        self.Conv_Wo = nn.Conv2d(C, C, filter_size, padding=2)
        self.Conv_Uo = nn.Conv2d(C, C, filter_size, padding=2)
        self.bo = Variable(torch.randn(H, device = device, dtype=dtype, requires_grad=True))
        
        self.Conv_Wc = nn.Conv2d(C, C, filter_size, padding=2)
        self.Conv_Uc = nn.Conv2d(C, C, filter_size, padding=2)
        self.bc = Variable(torch.randn(H, device = device, dtype=dtype, requires_grad=True))
        
        self.H0 = Variable(torch.randn(C, H, W, device = device, dtype=dtype, requires_grad=True))
        self.C0 = Variable(torch.randn(C, H, W, device = device, dtype=dtype, requires_grad=True))
        
                
    def forward(self, X):
        # put the forward pass here (iterating through the channels of X)
        # iteratively call AttentitiveConvLSTM_step
        # Return X', which is the features modified with the attention map
        Hprev = self.H0
        Cprev = self.C0
        Hprev = Hprev.unsqueeze(0)
        Cprev = Cprev.unsqueeze(0)
                
        for i in range (10):
            
            # Attention layer
            tanh_input = self.Conv_Wa(X) + self.Conv_Ua(Hprev) + self.ba
            tanh_output = self.Tanh(tanh_input)
            Z_t = self.Conv_Va(tanh_output)
#             print (i)
#             print (X.size())
#             print (Z_t.size())
            Z_t_reshaped = Z_t.view(Z_t.size()[0], 1, -1)
            A_t_initial = self.SoftMax_a(Z_t_reshaped)
            A_t = A_t_initial.view(-1, 1, H, W)
            A_t_expanded = A_t.expand(X.size())
            X_t = A_t_expanded * X

            # LSTM layer
            I_t = self.Sigmoid(self.Conv_Wi(X_t) + self.Conv_Ui(Hprev) + self.bi)
            F_t = self.Sigmoid(self.Conv_Wf(X_t) + self.Conv_Uf(Hprev) + self.bf)
            O_t = self.Sigmoid(self.Conv_Wo(X_t) + self.Conv_Uo(Hprev) + self.bo)
            G_t = self.Tanh(self.Conv_Wc(X_t) + self.Conv_Uc(Hprev) + self.bc)

            Cprev = Cprev * F_t + I_t * G_t
            Hprev = O_t * self.Tanh(Cprev)
        
        
        return X_t
        
        
#         tanh_input = F.conv2d(X, self.Wa, padding=2) + F.conv2d(Hprev, self.Ua, padding=2) + self.ba
#         Z_t = F.conv2d(nn.Tanh(tanh_input), self.Va, stride=1, padding=2)
#         print(Z_t.size())

        #         tanh_input1 = F.conv2d(X, (C, C, filter_size, filter_size), stride=1, padding=2)
#         tanh_input2 = F.conv2d(Hprev, (C, C, filter_size, filter_size), stride=1, padding=2)
#         tanh_input = tanh_input1 + tanh_input2 + self.ba

In [59]:
learning_rate = 2.2e-4
num_classes = 2

model = nn.Sequential(
    FeatureExtraction(),
    AttentiveConvLSTM(), # outputs attention map
   # ProduceSaliency(),
    MapToClass(),
)
if USE_GPU:
    model = model.cuda()

In [ ]:
optimizer = optim.Adam(model.parameters(), lr = learning_rate)
loss_arr = train(model, optimizer, 2)

Iteration 0, loss = 0.6921
train acc:
Got 1063 / 2100 correct (50.62)
cross acc:
Got 298 / 600 correct (49.67)
Iteration 5, loss = 0.6476
Iteration 10, loss = 0.7155
Iteration 15, loss = 0.6618
Iteration 20, loss = 0.6860
Iteration 25, loss = 0.6882
Iteration 30, loss = 0.6998
Iteration 35, loss = 0.6894
Iteration 40, loss = 0.6975
Iteration 45, loss = 0.6918
Iteration 50, loss = 0.7060
train acc:
Got 1055 / 2100 correct (50.24)
cross acc:
Got 304 / 600 correct (50.67)
Iteration 55, loss = 0.6890
Iteration 60, loss = 0.7255
Iteration 65, loss = 0.6974
Iteration 70, loss = 0.7264
Iteration 75, loss = 0.6937
Iteration 80, loss = 0.6843
Iteration 85, loss = 0.6935


In [ ]:
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
plt.plot(loss_arr)
plt.show()

In [ ]:
# saved_model = (torch.load("model_params.pt"))

# removed = list(saved_model.children())[:-1]
# model2= torch.nn.Sequential(*self.removed)


 #model2 = nn.Sequential(
#     model[0],
#     model[1],
#     ProduceSaliency()
#  )

# pretrained_state = (torch.load("model_params.pt"))
# model_state = model.state_dict()

# pretrained_state = { k:v for k,v in pretrained_state.iteritems() if k in model_state and v.size() == model_state[k].size() }
# model_state.update(pretrained_state)
# self._model.load_state_dict(model_state)

In [ ]:
for t, (x, y) in enumerate(loader_train):
#     model_out = model(Variable(x.float().type(dtypeFloat)))
    featureExtraction, attention = next(model.modules())[:2]
    feature_out = featureExtraction(Variable(x.float().type(dtypeFloat)))
    attention_out = attention(feature_out)
    
    attention_out_numpy = attention_out.data.cpu().numpy()
    print (attention_out_numpy.shape)
    img = np.mean(attention_out_numpy, axis=1)
    print (img.shape)
    
    
    
    
#     print(img.size())
#     img = img[4]
#     img = img.data.cpu().numpy()
#     img = img.reshape((H,W))
    imgplot = plt.imshow(img[0])
    img.show()
    break
# plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
# plt.rcParams['image.interpolation'] = 'nearest'
# plt.rcParams['image.cmap'] = 'gray'
# plt.plot(loss_arr)
# plt.show()